In [1]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

In [9]:
from WickContractions.wick.contract import *
from PyCorrelationMatrixManager.correlation_matrix import *

import os
import sys 

from src.utilities import *

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))

## Operators

First we have to define the operators

In [3]:
from src.op_elementals import *

aOps=[Operator([MesonBaryonSource(1,{'flavor': ['u','u','u'], 'x': 'x_0', 'gamma': '\\Gamma^B'},
                                    {'flavor': ['u','d'], 'x':'x_1', 'gamma': '\\Gamma^M'})])]

cOps=[Operator([MesonBaryonSink(1,{'flavor': ['u','u','u'], 'x': 'x_2', 'gamma': '\\Gamma^B'},
                                    {'flavor': ['d','u'], 'x':'x_3', 'gamma': '\\Gamma^M'})])]